# Lab | APIS

In this lab, you will collect historical data about the Nobel Prize winners using [this free and non-authenticated API](https://www.nobelprize.org/organization/developer-zone-2/). According to the documentation available [here](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1#/default/get_nobelPrizes). The base url is: "http://api.nobelprize.org/2.1/" followed by a string to specify what kind of information do you want to retrieve. The acceptable options are:

* nobelPrices
* nobelPrice/category/year
* laureates
* laureate/laureateID

# Getting the information using requests

Use the Python `requests`, and `json` libraries to obtain the information of ALL the Nobel Prizes. Make sure to verify that you get the proper status code (200).

The json outputs are simple plain text that need to be converted into the corresponding nested dictionary. Use the `.json()` method to cast the output into a Python dictionary.

Use the Pandas library to collect all the information into a Panda's DataFrame.

In [1]:
import requests
import json
import pandas as pd

url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
        print("All good!")
        print("==============")
        print("\n")
        data = response.json()["nobelPrizes"]
        data = pd.DataFrame(data)

All good!




# Processing the output

Process the Pandas DataFrame in order to have only the following columns:

- category
- dateAwarded (as DateTime in "yyyy-mm-dd" format)
- prizeAmount
- prizeAmountAdjusted
- Number_of_laureates
- motivation
- laureate_ids (as a list)

In [2]:
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
    prizes_list = response.json()['nobelPrizes']
    data = pd.DataFrame(prizes_list)
    nobel = pd.DataFrame()
    nobel['category'] = data['category'].str['en']
    nobel['dateAwarded'] = data['dateAwarded']
    nobel['prizeAmount'] = data['prizeAmount']
    nobel['prizeAmountAdjusted'] = data['prizeAmountAdjusted']
    nobel['Number_of_laureates'] = data['laureates'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    nobel['motivation'] = data['laureates'].apply(lambda x: x[0]['motivation']['en'] if isinstance(x, list) else None)
    nobel['laureate_ids'] = data['laureates'].apply(lambda x: [laureate['id'] for laureate in x] if isinstance(x, list) else [])
    
nobel

,category,dateAwarded,prizeAmount,prizeAmountAdjusted,Number_of_laureates,motivation,laureate_ids
0,Chemistry,1901-11-12,150782,10531894,1,in recognition of the extraordinary services h...,[160]
1,Literature,1901-11-14,150782,10531894,1,in special recognition of his poetic compositi...,[569]
2,Peace,1901-12-10,150782,10531894,2,for his humanitarian efforts to help wounded s...,"[462, 463]"
3,Physics,1901-11-12,150782,10531894,1,in recognition of the extraordinary services h...,[1]
4,Physiology or Medicine,1901-10-30,150782,10531894,1,"for his work on serum therapy, especially its ...",[293]
...,...,...,...,...,...,...,...
665,Economic Sciences,2023-10-09,11000000,11000000,1,for having advanced our understanding of women...,[1034]
666,Literature,2023-10-05,11000000,11000000,1,for his innovative plays and prose which give ...,[1032]
667,Peace,2023-10-06,11000000,11000000,1,for her fight against the oppression of women ...,[1033]
668,Physics,2023-10-03,11000000,11000000,3,for experimental methods that generate attosec...,"[1026, 1027, 1028]"


# Getting a Pandas DataFrame with the details of awarded authors/institutions

If you dive deeper and use the API to retrieve the details of some laureate_ids, you will notice that not allways the Nobel Prize was awarded to individuals. In some cases, the awards were given to institutions.

Get the unique ids from the previous datasets and prepare the following functions:

- get_name(laureate) ( it should return the english name 'fullName' of the individual or 'orgName' of the institution )

- get_gender(laureate) ( it should return the gender or 'Unknown' for individuals, and 'None' for institutions )

- get_birthdate(laureate) ( it should return the birthdate when it's avaialble or 'Unknown' otherwise )

- get_age(laureate) ( it should return the age of the awarded individual or 'Unknown' when it's not avaialble or for institutions )

- get_city(laureate) ( it should return the english name of the city when it's available or 'Unknown' otherwise )

- get_country(laureate) ( it should return the english name of the country when it's available or 'Unknown' otherwise )

- get_continent(laureate) ( it should return the english name of the continent when it's available or 'Unknown' otherwise )

- get_latitude(laureate) ( it should return the city's latitude when it's available or 'Unknown' otherwise )

- get_longitude(laureate) ( it should return the city's longitude
 when it's available or 'Unknown' otherwise )

Create the following dictionaries:

```python
laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}                        

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}
```

For each unique `laureate_id` of the previous DataFrame make an API call to get the details of the awarded individual/intitution and iterate of the previous dictionaries keys in order to add the corresponding information of each `laureate_id` in the empty lists of `laureates_dict`.

Finally, create a Pandas DataFrame named `laureates_df` using the `laureates_dict`.

In [130]:
import time
from tqdm import tqdm


ids = [int(item) for l in nobel['laureate_ids'].values for item in l]
unique_ids = set(ids)

def get_name(laureate):
    if 'fullName' in laureate:
        return laureate['fullName']['en']
    elif 'orgName' in laureate:
        return laureate['orgName']['en']
    return 'Unknown'

def get_gender(laureate):
    if 'gender' in laureate:
        return laureate['gender']
    elif 'orgName' in laureate:
        return None

def get_birthdate(laureate):

    birth_date = "Unknown"

    if 'birth' in laureate:
        return laureate['birth']['date']
    elif 'orgName' in laureate:
        return None
    return 'Unknown'

def get_age(laureate):

    birth_date = "Unknown"
    award_date = "Unknown"

    if 'birth' in laureate:
        birth_year = int(laureate['birth']['date'][0:4])
        award_date = int(laureate["nobelPrizes"][0]["awardYear"])
        age = award_date-birth_year
        return age
    elif 'orgName' in laureate:
        return 'Unknown'
    return 'Unknown'

def get_city(laureate):
    try:
        if 'birth' in laureate:
            return laureate["birth"]["place"]["city"]["en"]
        elif 'orgName' in laureate:
            return laureate['orgName']['en']
    except KeyError:
        return 'Unknown'
    return 'Unknown'

def get_country(laureate):
    try:
        if 'birth' in laureate:
            return laureate['birth']['place']['countryNow']['en']
        elif 'orgName' in laureate:
            return laureate['orgName']['en']
    except KeyError:
        return 'Unknown'
    return 'Unknown'
    
def get_continent(laureate):
    try:
        if 'birth' in laureate:
            return laureate['birth']['place']['continent']['en']
        elif 'orgName' in laureate:
            return laureate['orgName']['en']
    except KeyError:
        return 'Unknown'
    return 'Unknown'

def get_latitude(laureate):
    try:
        if 'birth' in laureate:
            return laureate['birth']['place']['cityNow']['latitude']
        elif 'orgName' in laureate:
            return laureate['orgName']['en']
    except KeyError:
        return 'Unknown'
    return 'Unknown'

def get_longitude(laureate):
    try:
        if 'birth' in laureate:
            return laureate['birth']['place']['cityNow']['longitude']
        elif 'orgName' in laureate:
            return laureate['orgName']['en']
    except KeyError:
        return 'Unknown'
    return 'Unknown'

laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}

for index, id in enumerate(tqdm(unique_ids)):

    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)

    if response.status_code == 200:

        laureate = response.json()

        laureates_dict["ID"].append(id)
        laureates_dict["Name"].append(get_name(laureate[0]))
        laureates_dict["Gender"].append(get_gender(laureate[0]))
        laureates_dict["Birth_date"].append(get_birthdate(laureate[0]))
        laureates_dict["Age"].append(get_age(laureate[0]))
        laureates_dict["City"].append(get_city(laureate[0]))
        laureates_dict["Country"].append(get_country(laureate[0]))
        laureates_dict["Continent"].append(get_continent(laureate[0]))
        laureates_dict["Latitude"].append(get_latitude(laureate[0]))
        laureates_dict["Longitude"].append(get_longitude(laureate[0]))

laureates_df = pd.DataFrame(laureates_dict)

laureates_df


100%|██████████| 992/992 [04:17<00:00,  3.85it/s]


,ID,Name,Gender,Birth_date,Age,City,Country,Continent,Latitude,Longitude
0,1,Wilhelm Conrad Röntgen,male,1845-03-27,56,Lennep,Germany,Europe,51.178742,7.189696
1,2,Hendrik Antoon Lorentz,male,1853-07-18,49,Arnhem,the Netherlands,Europe,51.984257,5.910857
2,3,Pieter Zeeman,male,1865-05-25,37,Zonnemaire,the Netherlands,Europe,51.713056,3.951111
3,4,Antoine Henri Becquerel,male,1852-12-15,51,Paris,France,Europe,48.860093,2.355954
4,5,Pierre Curie,male,1859-05-15,44,Paris,France,Europe,48.860093,2.355954
...,...,...,...,...,...,...,...,...,...,...
987,1030,Louis E. Brus,male,1943-00-00,80,"Cleveland, OH",USA,North America,41.496386,-81.710675
988,1031,Aleksey Yekimov,male,1945-00-00,78,Leningrad,Russia,Europe,59.956651,30.333547
989,1032,Jon Fosse,male,1959-09-29,64,Haugesund,Norway,Europe,59.410150,5.275511
990,1033,Narges Mohammadi,female,1972-04-21,51,Zanjan,Iran,Asia,36.666667,48.483333


# Country ranking

Get a ranking countries by the number of times that they had been awarded in any category.

In [131]:
laureates_df[["Country"]].value_counts().head(10)

Country       
USA               289
United Kingdom     89
Germany            84
France             61
Sweden             30
Russia             29
Japan              28
Poland             28
Canada             21
Italy              20
dtype: int64